# Deep Q Networks
Previously, we trained policies using policy gradient algorithms, which directly estimated the gradient of the returns for the policy and performed stochastic gradient ascent. In this section, we will now implement deep Q-learning, which does not explicitly optimize a policy, but simply infers the policy from a learned Q function that is trained via dynamic programming.

We will assume discrete action spaces for this notebook as to enable us to easily select actions that maximize the Q-function at given states.

In [1]:
# As usual, a bit of setup
import os
import shutil
import time
import torch
import numpy as np

import deeprl.infrastructure.pytorch_util as ptu

from deeprl.infrastructure.rl_trainer import RL_Trainer
from deeprl.infrastructure.trainers import PG_Trainer
from deeprl.infrastructure.trainers import DQN_Trainer

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def remove_folder(path):
    # check if folder exists
    if os.path.exists(path): 
        print("Clearing old results at {}".format(path))
        # remove if exists
        shutil.rmtree(path)
    else:
        print("Folder {} does not exist yet. No old results to delete".format(path))

In [2]:
dqn_base_args_dict = dict(
    env_name = 'LunarLander-v3', #@param 
    exp_name = 'test_dqn', #@param
    save_params = False, #@param {type: "boolean"}
    
    ## PDF will tell you how to set ep_len
    ## and discount for each environment
    ep_len = 200, #@param {type: "integer"}
    # discount = 0.95, #@param {type: "number"}

    # Training
    num_agent_train_steps_per_iter = 1, #@param {type: "integer"})
    num_critic_updates_per_agent_update = 1, #@param {type: "integer"}
  
    #@markdown Q-learning parameters
    double_q = False, #@param {type: "boolean"}

    # batches & buffers
    batch_size = 32, #@param {type: "integer"})
    batch_size_initial=1000,

    #@markdown logging
    video_log_freq = -1, #@param {type: "integer"}
    scalar_log_freq = 1000, #@param {type: "integer"}

    #@markdown gpu & run-time settings
    no_gpu = False, #@param {type: "boolean"}
    which_gpu = 0, #@param {type: "integer"}
    seed = 2, #@param {type: "integer"}
    logdir = 'test',
)

## DQN updates
Recall in Q-learning, we attempt to solve the optimal state-action values (which we refer to as Q-values $Q(s,a)$), by finding solutions to the Bellman equation given by
$$Q(s,a) = r(s,a) + \gamma \mathbb{E}_{s' \sim p(s'\vert s,a)}[\max_{a'}Q(s', a')].$$

Regular tabular Q-learning would take sample transitions $(s, a, r, s')$ and perform updates according to
$$Q(s,a) \leftarrow Q(s,a) + \alpha (r(s,a) + \gamma \max_{a'} Q(s', a') - Q(s,a)),$$
where $\alpha$ is a stepsize parameter.

This can be interpreted as updating $Q(s,a)$ by taking one gradient step on a squared Bellman error objective
$$(r(s, a) +\gamma \max_{a'} \tilde Q(s', a') - Q(s,a))^2,$$
where $\tilde Q$ is a copy of $Q$, but is not differentiated when taking the gradient step.

Adapting this update to the setting where we use a neural network with parameters $\theta$ to approximate $Q(s,a)$, we then train $\theta$ with the loss function 
$$\min_{\theta} \mathbb{E}_{s, a, s' \sim D} [L(Q_{\theta}(s,a), r(s,a) + \gamma \max_{a'} Q_{\tilde \theta}(s', a'))]$$
where $D$ is our replay buffer containing past transitions we've experienced, $L$ is some loss function capturing how far the predicted Q-values are from the target values, and $\tilde \theta$ are the target Q function parameters, which are usually a delayed copy of $\theta$ for stability reasons.

We note our previous policy gradient algorithms were _on-policy_ algoritms, which meant they updated the policy using only the data collected from the most recent policy, and discard all the data after using it just once. In contrast, DQN uses _off-policy_ updates by sampling data from all past interactions, allowing for data reuse over time.

Fill out the missing components for the basic Q-learning update in <code>critics/dqn_critic.py</code> (not including the double_q section).

In [84]:
#### Test DQN updates
dqn_args = dict(dqn_base_args_dict)

env_str = 'LunarLander'
dqn_args['env_name'] = '{}-v3'.format(env_str)
dqn_args['double_q'] = False
dqntrainer = DQN_Trainer(dqn_args)
dqnagent = dqntrainer.rl_trainer.agent
critic = dqnagent.critic

ob_dim = critic.ob_dim
ac_dim = 6
N = 5

np.random.seed(0)
obs = np.random.normal(size=(N, ob_dim))
acts = np.random.choice(ac_dim, size=(N,))
next_obs = np.random.normal(size=(N, ob_dim))
rewards = np.random.normal(size=N)
terminals = np.zeros(N)
terminals[0] = 1

first_weight_before = np.array(ptu.to_numpy(next(critic.q_net.parameters())))
print("Weight before update (first row)", first_weight_before[0])


loss = critic.update(obs, acts, next_obs, rewards, terminals)['Training Loss']
expected_loss = 0.9408444
loss_error = rel_error(loss, expected_loss)
print("Initial loss", loss)
print("Initial Loss Error", loss_error, "should be on the order of 1e-6 or lower")

for i in range(4):
    loss = critic.update(obs, acts, next_obs, rewards, terminals)['Training Loss']
    print(loss)

expected_loss = 0.7889254
loss_error = rel_error(loss, expected_loss)
print("Loss Error", loss_error, "should be on the order of 1e-6 or lower")


first_weight_after = np.array(ptu.to_numpy(next(critic.q_net.parameters())))
print("Weight after update (first row)", first_weight_after.shape)
# Test DQN gradient
print(first_weight_after[0])
weight_change_partial = first_weight_after[0] - first_weight_before[0]
expected_weight_change = np.array([-0.00491365, -0.00500049, -0.00499149, -0.00491229, -0.00490125,  0.00489534,
 -0.00282785, -0.00171614,  0.00485604])


updated_weight_error = rel_error(weight_change_partial, expected_weight_change)
print("Weight Update Error", updated_weight_error, "should be on the order of 1e-6 or lower")

########################
logging outputs to  test
########################
Using CPU for this assignment. There may be some bugs with using GPU that cause test cases to not match. You can uncomment the code below if you want to try using it.
LunarLander-v3
Weight before update (first row) [ 0.07646337 -0.07932475  0.09140956 -0.01702595  0.14239588  0.07935759
 -0.03831157 -0.2694876   0.07610479]
Initial loss 0.9408444
Initial Loss Error 8.831612855468697e-09 should be on the order of 1e-6 or lower
0.9007309
0.8621322
0.82467556
0.7889254
Loss Error 5.904878045285727e-09 should be on the order of 1e-6 or lower
Weight after update (first row) (64, 9)
[ 0.07154972 -0.08432525  0.08641808 -0.02193824  0.13749464  0.08425293
 -0.04113942 -0.27120373  0.08096083]
Weight Update Error 8.937585787696078e-07 should be on the order of 1e-6 or lower


/Users/sagarasanghavi/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Implement the missing components in the get_action method of <code>policies/argmax_policy.py</code> and the step_env method in <code>agents/dqn_agent.py</code> to allow our agent to interact with the environment.

In [45]:
### Test argmax policy
dqn_args = dict(dqn_base_args_dict)

env_str = 'LunarLander'
dqn_args['env_name'] = '{}-v3'.format(env_str)
dqn_args['double_q'] = False
dqntrainer = DQN_Trainer(dqn_args)
dqnagent = dqntrainer.rl_trainer.agent
actor = dqnagent.actor

ob_dim = critic.ob_dim
ac_dim = 6
N = 5

np.random.seed(0)
obs = np.random.normal(size=(N, ob_dim))

actions = actor.get_action(obs)
correct_actions = np.array([1, 0, 1, 0, 1])

assert np.all(correct_actions == actions)

########################
logging outputs to  test
########################
Using CPU for this assignment. There may be some bugs with using GPU that cause test cases to not match. You can uncomment the code below if you want to try using it.
LunarLander-v3


/Users/sagarasanghavi/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


We can now test our DQN implementation on the LunarLander environment. These experiments can take a while to run (over 10 minutes per seed) on CPU, so start early.

In [56]:
dqn_args = dict(dqn_base_args_dict)

env_str = 'LunarLander'
dqn_args['env_name'] = '{}-v3'.format(env_str)
dqn_args['double_q'] = False

# Delete all previous logs
remove_folder('logs/dqn/{}/vanilla_dqn'.format(env_str))

for seed in range(3):
    print("Running DQN experiment with seed", seed)
    dqn_args['seed'] = seed
    dqn_args['logdir'] = 'logs/dqn/{}/vanilla_dqn/seed{}'.format(env_str, seed)
    dqntrainer = DQN_Trainer(dqn_args)
    dqntrainer.run_training_loop()

Clearing old results at logs/dqn/LunarLander/vanilla_dqn
Running DQN experiment with seed 0
########################
logging outputs to  logs/dqn/LunarLander/vanilla_dqn/seed0
########################
Using CPU for this assignment. There may be some bugs with using GPU that cause test cases to not match. You can uncomment the code below if you want to try using it.
LunarLander-v3


********** Iteration 0 ************

Training agent...

Beginning logging procedure...
Timestep 1
mean reward (100 episodes) nan
best mean reward -inf
running time 0.001006
Train_EnvstepsSoFar : 1
TimeSinceStart : 0.0010058879852294922
Done logging...




/Users/sagarasanghavi/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




********** Iteration 1000 ************

Training agent...

Beginning logging procedure...
Timestep 1001
mean reward (100 episodes) -361.458927
best mean reward -inf
running time 0.663646
Train_EnvstepsSoFar : 1001
Train_AverageReturn : -361.4589273606985
TimeSinceStart : 0.6636459827423096
Done logging...




********** Iteration 2000 ************

Training agent...

Beginning logging procedure...
Timestep 2001
mean reward (100 episodes) -310.004133
best mean reward -inf
running time 4.109574
Train_EnvstepsSoFar : 2001
Train_AverageReturn : -310.0041333667857
TimeSinceStart : 4.109573841094971
Training Loss : 0.5019471645355225
Done logging...




********** Iteration 3000 ************

Training agent...

Beginning logging procedure...
Timestep 3001
mean reward (100 episodes) -310.660305
best mean reward -inf
running time 6.949341
Train_EnvstepsSoFar : 3001
Train_AverageReturn : -310.660304801111
TimeSinceStart : 6.9493408203125
Training Loss : 0.5492397546768188
Done logging...






Train_EnvstepsSoFar : 24001
Train_AverageReturn : -183.22477033921928
Train_BestReturn : -183.22477033921928
TimeSinceStart : 74.11091876029968
Training Loss : 0.6776131987571716
Done logging...




********** Iteration 25000 ************

Training agent...

Beginning logging procedure...
Timestep 25001
mean reward (100 episodes) -181.724345
best mean reward -181.724345
running time 77.561211
Train_EnvstepsSoFar : 25001
Train_AverageReturn : -181.7243454470348
Train_BestReturn : -181.7243454470348
TimeSinceStart : 77.5612108707428
Training Loss : 0.7930812835693359
Done logging...




********** Iteration 26000 ************

Training agent...

Beginning logging procedure...
Timestep 26001
mean reward (100 episodes) -180.220301
best mean reward -180.220301
running time 81.028885
Train_EnvstepsSoFar : 26001
Train_AverageReturn : -180.22030074503732
Train_BestReturn : -180.22030074503732
TimeSinceStart : 81.02888488769531
Training Loss : 1.0074446201324463
Done logging...




********** I

Train_EnvstepsSoFar : 45001
Train_AverageReturn : -149.8981531322144
Train_BestReturn : -149.46477256544978
TimeSinceStart : 153.97464394569397
Training Loss : 0.2759976387023926
Done logging...




********** Iteration 46000 ************

Training agent...

Beginning logging procedure...
Timestep 46001
mean reward (100 episodes) -150.156117
best mean reward -149.464773
running time 157.800271
Train_EnvstepsSoFar : 46001
Train_AverageReturn : -150.15611669220007
Train_BestReturn : -149.46477256544978
TimeSinceStart : 157.80027079582214
Training Loss : 4.0942888259887695
Done logging...




********** Iteration 47000 ************

Training agent...

Beginning logging procedure...
Timestep 47001
mean reward (100 episodes) -150.335903
best mean reward -149.464773
running time 162.017846
Train_EnvstepsSoFar : 47001
Train_AverageReturn : -150.33590301807683
Train_BestReturn : -149.46477256544978
TimeSinceStart : 162.01784563064575
Training Loss : 0.8969570398330688
Done logging...




*****

...

Train_EnvstepsSoFar : 412001
Train_AverageReturn : 115.94011008173494
Train_BestReturn : 154.28099773601443
TimeSinceStart : 1416.8993372917175
Training Loss : 0.3488525152206421
Done logging...




********** Iteration 413000 ************

Training agent...

Beginning logging procedure...
Timestep 413001
mean reward (100 episodes) 116.943736
best mean reward 154.280998
running time 1419.660407
Train_EnvstepsSoFar : 413001
Train_AverageReturn : 116.94373561465076
Train_BestReturn : 154.28099773601443
TimeSinceStart : 1419.6604070663452
Training Loss : 2.4622206687927246
Done logging...




********** Iteration 414000 ************

Training agent...

Beginning logging procedure...
Timestep 414001
mean reward (100 episodes) 122.909716
best mean reward 154.280998
running time 1422.390275
Train_EnvstepsSoFar : 414001
Train_AverageReturn : 122.9097156130925
Train_BestReturn : 154.28099773601443
TimeSinceStart : 1422.3902752399445
Training Loss : 1.606164574623108
Done logging...




*******

Train_EnvstepsSoFar : 433001
Train_AverageReturn : 151.6298229944743
Train_BestReturn : 154.28099773601443
TimeSinceStart : 1474.5339341163635
Training Loss : 0.09618035703897476
Done logging...




********** Iteration 434000 ************

Training agent...

Beginning logging procedure...
Timestep 434001
mean reward (100 episodes) 153.193028
best mean reward 154.280998
running time 1477.205372
Train_EnvstepsSoFar : 434001
Train_AverageReturn : 153.1930275809381
Train_BestReturn : 154.28099773601443
TimeSinceStart : 1477.2053723335266
Training Loss : 2.216078042984009
Done logging...




********** Iteration 435000 ************

Training agent...

Beginning logging procedure...
Timestep 435001
mean reward (100 episodes) 159.145950
best mean reward 159.145950
running time 1479.874536
Train_EnvstepsSoFar : 435001
Train_AverageReturn : 159.14594958299395
Train_BestReturn : 159.14594958299395
TimeSinceStart : 1479.8745362758636
Training Loss : 0.22744882106781006
Done logging...




******

Train_EnvstepsSoFar : 454001
Train_AverageReturn : 147.72136412578413
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1530.8314652442932
Training Loss : 0.08983515948057175
Done logging...




********** Iteration 455000 ************

Training agent...

Beginning logging procedure...
Timestep 455001
mean reward (100 episodes) 147.857998
best mean reward 161.944809
running time 1533.480659
Train_EnvstepsSoFar : 455001
Train_AverageReturn : 147.8579983535985
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1533.4806592464447
Training Loss : 3.9369094371795654
Done logging...




********** Iteration 456000 ************

Training agent...

Beginning logging procedure...
Timestep 456001
mean reward (100 episodes) 145.908502
best mean reward 161.944809
running time 1536.164843
Train_EnvstepsSoFar : 456001
Train_AverageReturn : 145.9085020744729
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1536.164843082428
Training Loss : 0.20499911904335022
Done logging...




******

Train_EnvstepsSoFar : 475001
Train_AverageReturn : 109.07073747742672
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1588.0679211616516
Training Loss : 1.6317424774169922
Done logging...




********** Iteration 476000 ************

Training agent...

Beginning logging procedure...
Timestep 476001
mean reward (100 episodes) 108.157606
best mean reward 161.944809
running time 1590.824520
Train_EnvstepsSoFar : 476001
Train_AverageReturn : 108.15760568752131
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1590.8245203495026
Training Loss : 0.20396649837493896
Done logging...




********** Iteration 477000 ************

Training agent...

Beginning logging procedure...
Timestep 477001
mean reward (100 episodes) 100.193685
best mean reward 161.944809
running time 1593.555040
Train_EnvstepsSoFar : 477001
Train_AverageReturn : 100.1936852575259
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1593.5550401210785
Training Loss : 0.18437299132347107
Done logging...




****

Train_EnvstepsSoFar : 496001
Train_AverageReturn : 70.37033736636005
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1646.2972462177277
Training Loss : 7.371368408203125
Done logging...




********** Iteration 497000 ************

Training agent...

Beginning logging procedure...
Timestep 497001
mean reward (100 episodes) 66.434760
best mean reward 161.944809
running time 1648.955200
Train_EnvstepsSoFar : 497001
Train_AverageReturn : 66.4347597544451
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1648.955199956894
Training Loss : 1.1901475191116333
Done logging...




********** Iteration 498000 ************

Training agent...

Beginning logging procedure...
Timestep 498001
mean reward (100 episodes) 64.204122
best mean reward 161.944809
running time 1651.603387
Train_EnvstepsSoFar : 498001
Train_AverageReturn : 64.20412236253841
Train_BestReturn : 161.94480942757139
TimeSinceStart : 1651.6033871173859
Training Loss : 0.20250026881694794
Done logging...




********** I

In [57]:
### Visualize vanilla DQN results on Lunar Lander
%load_ext tensorboard
%tensorboard --logdir logs/dqn/LunarLander/vanilla_dqn

## Double DQN
One potential issue with learning our Q functions with bootstrapping is _maximization bias_, where the learned Q-values tend to overestimate the actual expected future returns. The main idea is that when there is estimation error in the next state's Q-values, even if the values were correct on average, picking the action with the maximum Q-value would tend to select one where the value is overestimated. This overoptimistic value would then also get propagated via the Bellman backups to other states and actions, and can potentially slow down learning.

Double DQN (https://arxiv.org/abs/1509.06461) proposes a simple solution to alleviate this _maximization bias_. Instead of taking the next action that maximizes the target network's Q-value, it selects the action to maximize the _current_ Q function at the next state, and then takes the target network's estimate of that action's value. 

Implement the double DQN target value in the update method in <code>critics/dqn_critic.py</code>.

In [83]:
#### Test DQN target value with double Q
dqn_args = dict(dqn_base_args_dict)

env_str = 'LunarLander'
dqn_args['env_name'] = '{}-v3'.format(env_str)
dqn_args['double_q'] = True
dqntrainer = DQN_Trainer(dqn_args)
dqnagent = dqntrainer.rl_trainer.agent
critic = dqnagent.critic

ob_dim = critic.ob_dim
ac_dim = 6
N = 5

np.random.seed(0)
obs = np.random.normal(size=(N, ob_dim))
acts = np.random.choice(ac_dim, size=(N,))
next_obs = np.random.normal(size=(N, ob_dim))
rewards = np.random.normal(size=N)
terminals = np.zeros(N)
terminals[0] = 1

first_weight_before = np.array(ptu.to_numpy(next(critic.q_net.parameters())))
print("Weight before update (first row)", first_weight_before[0])


loss = critic.update(obs, acts, next_obs, rewards, terminals)['Training Loss']
expected_loss = 0.93894196
loss_error = rel_error(loss, expected_loss)
print("Initial loss", loss)
print("Initial Loss Error", loss_error, "should be on the order of 1e-6 or lower")

for i in range(4):
    loss = critic.update(obs, acts, next_obs, rewards, terminals)['Training Loss']
    print(loss)

expected_loss = 0.7871182
loss_error = rel_error(loss, expected_loss)
print("Loss Error", loss_error, "should be on the order of 1e-6 or lower")


first_weight_after = np.array(ptu.to_numpy(next(critic.q_net.parameters())))
print("Weight after update (first row)", first_weight_after.shape)
# Test DQN gradient
print(first_weight_after[0])
weight_change_partial = first_weight_after[0] - first_weight_before[0]
print(weight_change_partial)
expected_weight_change = np.array([-0.0049137, -0.00500057, -0.00499138, -0.00491226, -0.00490116,  0.00489506,
 -0.00284088, -0.00171939,  0.00485736])


updated_weight_error = rel_error(weight_change_partial, expected_weight_change)
print("Weight Update Error", updated_weight_error, "should be on the order of 1e-6 or lower")

########################
logging outputs to  test
########################
Using CPU for this assignment. There may be some bugs with using GPU that cause test cases to not match. You can uncomment the code below if you want to try using it.
LunarLander-v3
Weight before update (first row) [ 0.07646337 -0.07932475  0.09140956 -0.01702595  0.14239588  0.07935759
 -0.03831157 -0.2694876   0.07610479]
Initial loss 0.93894196
Initial Loss Error 2.3609519617425807e-09 should be on the order of 1e-6 or lower
0.8988525
0.8602768
0.8228415
0.7871182
Loss Error 2.231287016432748e-09 should be on the order of 1e-6 or lower
Weight after update (first row) (64, 9)
[ 0.07154967 -0.08432532  0.08641818 -0.02193821  0.13749473  0.08425266
 -0.04115245 -0.27120697  0.08096215]
[-0.0049137  -0.00500057 -0.00499138 -0.00491226 -0.00490116  0.00489506
 -0.00284088 -0.00171939  0.00485736]
Weight Update Error 1.3418982564751781e-06 should be on the order of 1e-6 or lower


/Users/sagarasanghavi/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


We can now also run some experiments on LunarLander with Double DQN. You may be able to see that double DQN performs slightly better and more stably, but as there is very high variance, dont' worry if you do not.

In [85]:
# Run with double DQN
dqn_args = dict(dqn_base_args_dict)

env_str = 'LunarLander'
dqn_args['env_name'] = '{}-v3'.format(env_str)
dqn_args['double_q'] = True

# Delete all previous logs
remove_folder('logs/dqn/{}/double_dqn'.format(env_str))

for seed in range(3):
    print("Running DQN experiment with seed", seed)
    dqn_args['seed'] = seed
    dqn_args['logdir'] = 'logs/dqn/{}/double_dqn/seed{}'.format(env_str, seed)
    dqntrainer = DQN_Trainer(dqn_args)
    dqntrainer.run_training_loop()

Folder logs/dqn/LunarLander/double_dqn does not exist yet. No old results to delete
Running DQN experiment with seed 0
########################
logging outputs to  logs/dqn/LunarLander/double_dqn/seed0
########################
Using CPU for this assignment. There may be some bugs with using GPU that cause test cases to not match. You can uncomment the code below if you want to try using it.
LunarLander-v3


********** Iteration 0 ************

Training agent...

Beginning logging procedure...
Timestep 1
mean reward (100 episodes) nan
best mean reward -inf
running time 0.000730
Train_EnvstepsSoFar : 1
TimeSinceStart : 0.0007300376892089844
Done logging...




/Users/sagarasanghavi/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))




********** Iteration 1000 ************

Training agent...

Beginning logging procedure...
Timestep 1001
mean reward (100 episodes) -361.458927
best mean reward -inf
running time 0.657439
Train_EnvstepsSoFar : 1001
Train_AverageReturn : -361.4589273606985
TimeSinceStart : 0.6574389934539795
Done logging...




********** Iteration 2000 ************

Training agent...

Beginning logging procedure...
Timestep 2001
mean reward (100 episodes) -317.658504
best mean reward -inf
running time 4.847034
Train_EnvstepsSoFar : 2001
Train_AverageReturn : -317.6585039542492
TimeSinceStart : 4.847033977508545
Training Loss : 0.3789442181587219
Done logging...




********** Iteration 3000 ************

Training agent...

Beginning logging procedure...
Timestep 3001
mean reward (100 episodes) -298.225730
best mean reward -inf
running time 8.871307
Train_EnvstepsSoFar : 3001
Train_AverageReturn : -298.2257298425541
TimeSinceStart : 8.871306896209717
Training Loss : 0.24803029000759125
Done logging...




********** Iteration 24000 ************

Training agent...

Beginning logging procedure...
Timestep 24001
mean reward (100 episodes) -212.727138
best mean reward -212.727138
running time 109.342728
Train_EnvstepsSoFar : 24001
Train_AverageReturn : -212.7271384089546
Train_BestReturn : -212.7271384089546
TimeSinceStart : 109.34272789955139
Training Loss : 0.836423397064209
Done logging...




********** Iteration 25000 ************

Training agent...

Beginning logging procedure...
Timestep 25001
mean reward (100 episodes) -210.460493
best mean reward -210.460493
running time 114.993388
Train_EnvstepsSoFar : 25001
Train_AverageReturn : -210.46049303442564
Train_BestReturn : -210.46049303442564
TimeSinceStart : 114.99338793754578
Training Loss : 1.9687288999557495
Done logging...




********** Iteration 26000 ************

Training agent...

Beginning logging procedure...
Timestep 26001
mean reward (100 episodes) -210.516870
best mean reward -210.460493
running time 120.341567
Train_E



********** Iteration 45000 ************

Training agent...

Beginning logging procedure...
Timestep 45001
mean reward (100 episodes) -178.152212
best mean reward -178.152212
running time 214.358650
Train_EnvstepsSoFar : 45001
Train_AverageReturn : -178.1522118803098
Train_BestReturn : -178.1522118803098
TimeSinceStart : 214.35864996910095
Training Loss : 0.4385693073272705
Done logging...




********** Iteration 46000 ************

Training agent...

Beginning logging procedure...
Timestep 46001
mean reward (100 episodes) -171.024686
best mean reward -171.024686
running time 219.908688
Train_EnvstepsSoFar : 46001
Train_AverageReturn : -171.0246862056383
Train_BestReturn : -171.0246862056383
TimeSinceStart : 219.9086878299713
Training Loss : 3.0320754051208496
Done logging...




********** Iteration 47000 ************

Training agent...

Beginning logging procedure...
Timestep 47001
mean reward (100 episodes) -169.350078
best mean reward -169.350078
running time 224.242533
Train_Env

...

Train_EnvstepsSoFar : 413001
Train_AverageReturn : 133.30392573762677
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1886.878389120102
Training Loss : 0.48190179467201233
Done logging...




********** Iteration 414000 ************

Training agent...

Beginning logging procedure...
Timestep 414001
mean reward (100 episodes) 135.634789
best mean reward 142.789345
running time 1890.828728
Train_EnvstepsSoFar : 414001
Train_AverageReturn : 135.63478919531053
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1890.828727722168
Training Loss : 0.2411002218723297
Done logging...




********** Iteration 415000 ************

Training agent...

Beginning logging procedure...
Timestep 415001
mean reward (100 episodes) 135.733225
best mean reward 142.789345
running time 1894.968113
Train_EnvstepsSoFar : 415001
Train_AverageReturn : 135.73322500173077
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1894.9681129455566
Training Loss : 0.14736312627792358
Done logging...




********

Train_EnvstepsSoFar : 434001
Train_AverageReturn : 126.4744099109083
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1969.6733059883118
Training Loss : 1.5191841125488281
Done logging...




********** Iteration 435000 ************

Training agent...

Beginning logging procedure...
Timestep 435001
mean reward (100 episodes) 132.309693
best mean reward 142.789345
running time 1973.314711
Train_EnvstepsSoFar : 435001
Train_AverageReturn : 132.30969280799002
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1973.3147110939026
Training Loss : 2.764617443084717
Done logging...




********** Iteration 436000 ************

Training agent...

Beginning logging procedure...
Timestep 436001
mean reward (100 episodes) 132.126307
best mean reward 142.789345
running time 1976.928221
Train_EnvstepsSoFar : 436001
Train_AverageReturn : 132.12630748632992
Train_BestReturn : 142.7893445951395
TimeSinceStart : 1976.92822098732
Training Loss : 1.7505033016204834
Done logging...




********** I

Train_EnvstepsSoFar : 455001
Train_AverageReturn : 149.68419810478292
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2048.4714210033417
Training Loss : 0.1774071902036667
Done logging...




********** Iteration 456000 ************

Training agent...

Beginning logging procedure...
Timestep 456001
mean reward (100 episodes) 142.348715
best mean reward 152.159215
running time 2052.645479
Train_EnvstepsSoFar : 456001
Train_AverageReturn : 142.34871533808527
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2052.645478963852
Training Loss : 0.3210744857788086
Done logging...




********** Iteration 457000 ************

Training agent...

Beginning logging procedure...
Timestep 457001
mean reward (100 episodes) 138.145982
best mean reward 152.159215
running time 2056.828600
Train_EnvstepsSoFar : 457001
Train_AverageReturn : 138.14598155385536
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2056.8285999298096
Training Loss : 0.20742270350456238
Done logging...




*****

Train_EnvstepsSoFar : 476001
Train_AverageReturn : 133.75172778044828
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2128.074028968811
Training Loss : 0.3200947940349579
Done logging...




********** Iteration 477000 ************

Training agent...

Beginning logging procedure...
Timestep 477001
mean reward (100 episodes) 133.148790
best mean reward 152.159215
running time 2131.694787
Train_EnvstepsSoFar : 477001
Train_AverageReturn : 133.14878984269617
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2131.694786787033
Training Loss : 0.2311696857213974
Done logging...




********** Iteration 478000 ************

Training agent...

Beginning logging procedure...
Timestep 478001
mean reward (100 episodes) 129.001348
best mean reward 152.159215
running time 2135.326839
Train_EnvstepsSoFar : 478001
Train_AverageReturn : 129.00134808316
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2135.3268389701843
Training Loss : 0.3433936536312103
Done logging...




**********

Train_EnvstepsSoFar : 497001
Train_AverageReturn : 133.03615477807654
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2204.8352420330048
Training Loss : 0.644425094127655
Done logging...




********** Iteration 498000 ************

Training agent...

Beginning logging procedure...
Timestep 498001
mean reward (100 episodes) 132.500988
best mean reward 152.159215
running time 2208.465910
Train_EnvstepsSoFar : 498001
Train_AverageReturn : 132.50098807127966
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2208.4659099578857
Training Loss : 2.8511452674865723
Done logging...




********** Iteration 499000 ************

Training agent...

Beginning logging procedure...
Timestep 499001
mean reward (100 episodes) 136.360393
best mean reward 152.159215
running time 2212.102291
Train_EnvstepsSoFar : 499001
Train_AverageReturn : 136.36039334839845
Train_BestReturn : 152.15921525328358
TimeSinceStart : 2212.102290868759
Training Loss : 1.2850500345230103
Done logging...




In [86]:
### Visualize all DQN results on Lunar Lander
%load_ext tensorboard
%tensorboard --logdir logs/dqn/LunarLander/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
